In [ ]:
import pandas as pd
import yfinance as yf
from gnews import GNews
from tqdm import tqdm
import time
import datetime
from dateutil.relativedelta import relativedelta
import random
import os

# ================= 配置 =================
OUTPUT_FILE = "gold_news_10years.csv"
GLOBAL_START = datetime.date(2020, 1, 1)
GLOBAL_END = datetime.date(2025, 12, 31)

# 黄金受什么影响？我们就搜什么
KEYWORDS = [
    "Gold price", "Federal Reserve rate", "Inflation CPI", 
    "Geopolitical tension", "US Dollar index", "Recession fears"
]

def fetch_macro_news():
    print(f"🕷️ 启动黄金宏观爬虫 ({GLOBAL_START} - {GLOBAL_END})...")
    all_news = []
    
    current_date = GLOBAL_START
    
    # 进度条
    total_months = (GLOBAL_END.year - GLOBAL_START.year) * 12 + (GLOBAL_END.month - GLOBAL_START.month)
    pbar = tqdm(total=total_months, desc="Fetching")

    while current_date < GLOBAL_END:
        next_month = current_date + relativedelta(months=1)
        
        try:
            # 每月重置 GNews
            google_news = GNews(
                language='en', country='US', max_results=50,
                start_date=(current_date.year, current_date.month, current_date.day),
                end_date=(next_month.year, next_month.month, next_month.day)
            )
            
            for kw in KEYWORDS:
                try:
                    resp = google_news.get_news(kw)
                    for art in resp:
                        title = art.get('title', '')
                        pub = art.get('published date', '')
                        
                        if len(title) < 10: continue
                        
                        try:
                            # 统一日期格式
                            dt = pd.to_datetime(pub).strftime("%Y-%m-%d")
                        except:
                            dt = current_date.strftime("%Y-%m-%d")
                            
                        all_news.append({
                            'Date': dt,
                            'Topic': kw,
                            'Headline': title
                        })
                    time.sleep(0.3)
                except: pass
                
        except Exception as e:
            print(f"Month Error: {e}")
            
        current_date = next_month
        pbar.update(1)
        
        # 每年存一次档
        if current_date.month == 1:
            pd.DataFrame(all_news).drop_duplicates(subset=['Headline']).to_csv(f"gold_news_temp_{current_date.year}.csv", index=False)

    pbar.close()
    
    df = pd.DataFrame(all_news).drop_duplicates(subset=['Headline']).sort_values('Date')
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"✅ 完成！共爬取 {len(df)} 条宏观新闻。")

if __name__ == "__main__":
    fetch_macro_news()